In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold
import umap
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris

c:\Users\waqar\AppData\Local\pypoetry\Cache\virtualenvs\inverse-projection-j9xKauoB-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create 3D Gaussians

In [2]:
dim = 3
n_gauss = 6
n_pts_per_gauss = 300
np.random.seed(5)

# centers = np.zeros((n_gauss,dim))
# for i in range(1,n_gauss):
#     centers[i] = np.random.randint(0,2,3)
centers = np.random.uniform(-1,1,size=(n_gauss,3))
    
print(centers)

cov_m = [np.diag([0.01 for i in range(dim)]),np.diag([0.01 if i%2 !=0 else 0.01 for i in range(dim)])]

D = np.zeros((n_pts_per_gauss*n_gauss,dim))
c = np.zeros(n_pts_per_gauss*n_gauss)      # storage for labels
for i in range(n_gauss):
    k = np.random.randint(0,2,1)[0]
    D[i*n_pts_per_gauss:(i+1)*n_pts_per_gauss] = np.random.multivariate_normal(centers[i],cov_m[k],n_pts_per_gauss)
    c[i*n_pts_per_gauss:(i+1)*n_pts_per_gauss] = i 
D = (D-np.min(D,axis=0))/(np.max(D,axis=0)-np.min(D,axis=0))
print(D.shape)
print(c.shape)

[[-0.55601366  0.74146461 -0.58656169]
 [ 0.83722182 -0.02317762  0.22348773]
 [ 0.53181571  0.03683598 -0.406399  ]
 [-0.62455754 -0.83851746  0.47688059]
 [-0.11738155 -0.68338026  0.75987406]
 [-0.45182708 -0.17152996 -0.40784013]]
(1800, 3)
(1800,)


In [3]:
%matplotlib qt

# colors = ['r', 'g', 'b']  # Red, Green, Blue
colors = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#00FFFF', '#FF00FF', '#000000']
# Create a figure and 3D axis
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(projection='3d')

# Define colors for each Gaussian distribution

# Loop through each Gaussian to plot points with corresponding color
for i in range(n_gauss):
    ax.scatter(D[c == i, 0], D[c == i, 1], D[c == i, 2], color=colors[i], label=f'Gaussian {i+1}')
    # ax.scatter(D[:,0], D[:,1], D[:,2], c=c)

# Set labels and title
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
ax.set_title('3D Scatter Plot of Data Points from Three Gaussian Distributions')

# Add a legend
ax.legend()

# Show the plot
plt.show()

## Project 3D to 2D using T-SNE

In [4]:
# t_sne = manifold.TSNE(
#     n_components=2,
#     perplexity=7,
#     init="random",
#     random_state=0,
# )

# S = t_sne.fit_transform(D)

In [5]:
t_sne = umap.UMAP(
    n_components=2,     # Targeting 2D projection
    n_neighbors=10,     # Similar to t-SNE perplexity
    min_dist=0.1,       # Controls the compactness of the clusters
    init="random",
    random_state=0,
)


# Apply UMAP on the 3D Gaussian data `D`
S = t_sne.fit_transform(D)

c:\Users\waqar\AppData\Local\pypoetry\Cache\virtualenvs\inverse-projection-j9xKauoB-py3.12\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\waqar\AppData\Local\pypoetry\Cache\virtualenvs\inverse-projection-j9xKauoB-py3.12\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


# Plot T-SNE Output (3D --> 2D )

In [6]:
# Plotting the t-SNE results with the same color scheme
%matplotlib qt

# colors = ['r', 'g', 'b','']  # Red, Green, Blue
plt.figure(figsize=(10, 8))
for i in range(n_gauss):
    plt.scatter(S[c == i, 0], S[c == i, 1], color=colors[i], label=f'Gaussian {i+1}')
    # plt.scatter(S[c == i, 0], S[c == i, 1], label=f'Gaussian {i+1}')

plt.title('t-SNE Visualization of 3D Gaussian Distributions into 2D')
plt.legend()
plt.grid(True)
plt.show()

## Create a 2D Grid for Jacobian Calculation

In [7]:
# Define min and max values
x_min, x_max = np.min(S[:, 0]), np.max(S[:, 0])
y_min, y_max = np.min(S[:, 1]), np.max(S[:, 1])
print(x_min, x_max)
print(y_min, y_max)

-3.6020608 16.442839
-7.361164 18.883371


In [8]:
# Define grid resolution
num_grid_points = 100

# Generate grid
x_vals = np.linspace(x_min, x_max, num_grid_points)
y_vals = np.linspace(y_min, y_max, num_grid_points)
xx, yy = np.meshgrid(x_vals, y_vals)
print(yy.shape)
print(y_vals.shape)

(100, 100)
(100,)


In [9]:
print(x_vals[:3])
print(x_vals[-3:])
print(y_vals[:3])
print(y_vals[-3:])

[-3.6020608 -3.3995872 -3.1971133]
[16.037891 16.240366 16.442839]
[-7.361164  -7.096068  -6.8309717]
[18.353178 18.618275 18.883371]


In [10]:
yy

array([[-7.361164 , -7.361164 , -7.361164 , ..., -7.361164 , -7.361164 ,
        -7.361164 ],
       [-7.096068 , -7.096068 , -7.096068 , ..., -7.096068 , -7.096068 ,
        -7.096068 ],
       [-6.8309717, -6.8309717, -6.8309717, ..., -6.8309717, -6.8309717,
        -6.8309717],
       ...,
       [18.353178 , 18.353178 , 18.353178 , ..., 18.353178 , 18.353178 ,
        18.353178 ],
       [18.618275 , 18.618275 , 18.618275 , ..., 18.618275 , 18.618275 ,
        18.618275 ],
       [18.883371 , 18.883371 , 18.883371 , ..., 18.883371 , 18.883371 ,
        18.883371 ]], dtype=float32)

In [11]:
grid_points = np.c_[xx.ravel(), yy.ravel()]

In [12]:
grid_points.shape

(10000, 2)

In [13]:
%matplotlib qt

plt.figure(figsize=(10, 8))
# Visualize the grid on top of the t-SNE data
plt.scatter(S[:, 0], S[:, 1], c='blue', s=10, label="t-SNE Output")
plt.scatter(xx, yy, c='red', s=5, label="Grid Points")
plt.title("2D t-SNE Output with Grid Points")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
# plt.grid(True)
plt.show()

## Define the Inverse Projection

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

# Define the MLP inverse_model
class NNinv(nn.Module):
    def __init__(self, input_size, output_size):
        super(NNinv, self).__init__()
        
        # Define the layers
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),  # Input to first hidden layer
            nn.ReLU(),
            nn.Linear(64, 128),  # First hidden layer to second hidden layer
            nn.ReLU(),
            nn.Linear(128, 256),  # Second hidden layer to third hidden layer
            nn.ReLU(),
            nn.Linear(256, 512),  # Third hidden layer to fourth hidden layer
            nn.ReLU(),
            nn.Linear(512, output_size),  # Fifth hidden layer to output
            nn.Sigmoid()  # Output layer with sigmoid activation
        )
    
    def forward(self, x):
        return self.layers(x)


In [15]:
X_train, X_test, y_train, y_test, c_train, c_test = train_test_split(S, D,c, test_size=0.33, random_state=42, stratify=c)

In [16]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(c_train.shape)
print(c_test.shape)

(1206, 2)
(594, 2)
(1206, 3)
(594, 3)
(1206,)
(594,)


In [17]:
print(X_train[0])
print(y_train[0])

[13.465107 -4.352659]
[0.24799929 0.85297565 0.13417306]


In [18]:
# import numpy as np

# Assuming c_train is your array
unique_values, counts = np.unique(c_test, return_counts=True)

# Display the unique values and their counts
for value, count in zip(unique_values, counts):
    print(f"Class {value}: {count} occurrences")


Class 0.0: 99 occurrences
Class 1.0: 99 occurrences
Class 2.0: 99 occurrences
Class 3.0: 99 occurrences
Class 4.0: 99 occurrences
Class 5.0: 99 occurrences


In [19]:
# Example usage
input_size = 2  # Example input size (can be changed)
output_size = dim   # Binary classification (sigmoid output for single output)

# Create DataLoader for batch processing
batch_size = 64
t_X_train = torch.tensor(X_train)
t_y_train = torch.tensor(y_train)
dataset = TensorDataset(t_X_train, t_y_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Instantiate the inverse_model, loss function, and optimizer
inverse_model = NNinv(input_size, output_size)
loss_fn = nn.L1Loss()  # Mean Absolute Error (MAE)
optimizer = optim.Adam(inverse_model.parameters(), lr=0.001)

# Number of epochs to train
num_epochs = 20

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, targets) in enumerate(dataloader):
        # Forward pass
        outputs = inverse_model(inputs)
        loss = loss_fn(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    # Print the average loss for the epoch
    avg_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

print("Training complete.")

Epoch [1/20], Loss: 0.1244
Epoch [2/20], Loss: 0.0597
Epoch [3/20], Loss: 0.0502
Epoch [4/20], Loss: 0.0422
Epoch [5/20], Loss: 0.0391
Epoch [6/20], Loss: 0.0369
Epoch [7/20], Loss: 0.0367
Epoch [8/20], Loss: 0.0346
Epoch [9/20], Loss: 0.0336
Epoch [10/20], Loss: 0.0320
Epoch [11/20], Loss: 0.0314
Epoch [12/20], Loss: 0.0320
Epoch [13/20], Loss: 0.0317
Epoch [14/20], Loss: 0.0306
Epoch [15/20], Loss: 0.0304
Epoch [16/20], Loss: 0.0305
Epoch [17/20], Loss: 0.0309
Epoch [18/20], Loss: 0.0310
Epoch [19/20], Loss: 0.0315
Epoch [20/20], Loss: 0.0296
Training complete.


In [20]:
t_X_test = torch.tensor(X_test)
t_y_test = torch.tensor(y_test)
outputs_test = inverse_model(t_X_test)
loss_test = loss_fn(outputs_test, t_y_test)
print(loss_test/y_test.shape[0])

tensor(5.2184e-05, dtype=torch.float64, grad_fn=<DivBackward0>)


In [21]:
t_y_test.shape

torch.Size([594, 3])

# Visualizing Inverse Projection

In [22]:
%matplotlib qt

# Create a figure and 3D axis
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(projection='3d')

# Define colors for each Gaussian distribution
# colors = ['r', 'g', 'b']  # Red, Green, Blue


output_fin = outputs_test.detach().numpy()
# Loop through each Gaussian to plot points with corresponding color
for i in range(n_gauss):
    ax.scatter(t_y_test[c_test == i, 0], t_y_test[c_test == i, 1], t_y_test[c_test == i, 2], color=colors[i], label=f'Actual_Gaussian {i+1}')
    # ax.scatter(output_fin[c_test == i, 0], output_fin[c_test == i, 1], output_fin[c_test == i, 2], color='orange', label=f'Predicted_Gaussian {i+1}')

ax.scatter(output_fin[:, 0], output_fin[:, 1], output_fin[:, 2], color='orange', label=f'Predicted_Gaussians')

# Set labels and title
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
ax.set_title('TSNE \n Actual Vs Prediction')

# Add a legend
ax.legend()

# Show the plot
plt.show()

## Validating 2D projection

In [23]:
def generate_spread_points(S, labels, num_new_points_per_cluster=5, spread_factor=0.5):
    """
    Generate new points around the spread of the Gaussian clusters in 2D t-SNE space.
    
    Parameters:
    S (np.array): 2D t-SNE points (original).
    labels (np.array): Labels for the original points, corresponding to Gaussian clusters.
    num_new_points_per_cluster (int): Number of new points to generate per Gaussian cluster.
    spread_factor (float): Spread factor controlling the variance of new points.
    
    Returns:
    new_points (np.array): Newly generated points spread around each cluster.
    new_labels (np.array): Labels corresponding to the new points.
    """
    new_points = []
    new_labels = []
    
    # Get the unique labels (each label corresponds to one Gaussian)
    unique_labels = np.unique(labels)

    for label in unique_labels:
        # Get the points that belong to the current Gaussian cluster
        cluster_points = S[labels == label]
        
        # Calculate covariance matrix for the current cluster
        cluster_cov = np.cov(cluster_points.T)

        for _ in range(num_new_points_per_cluster):
            # Randomly choose a point within the cluster
            random_point = cluster_points[np.random.randint(len(cluster_points))]
            
            # Generate a random offset using the covariance matrix to create a spread
            offset = np.random.multivariate_normal([0, 0], spread_factor * cluster_cov)

            # Add the offset to the selected random point to create a new point
            new_point = random_point + offset
            new_points.append(new_point)
            new_labels.append(label)  # Assign the same label as the original points
    
    return np.array(new_points), np.array(new_labels)


In [24]:
new_points, new_labels = generate_spread_points(S, c, num_new_points_per_cluster=20, spread_factor=0.3)

# Plot new points

In [25]:
markers = ['$G1$', '$G2$', '$G3$', '$G4$', '$G5$', '$G6$', '$G7$']

plt.figure(figsize=(10, 8))
for i in range(n_gauss):
    plt.scatter(S[c == i, 0], S[c == i, 1], color=colors[i], label=f'Gaussian {i+1}')

    # Plot new points
    plt.scatter(new_points[new_labels == i, 0], new_points[new_labels == i, 1], color=colors[i], marker = markers[i] , s = 100, edgecolors='black', label= f'New Points_Gaussian {i+1}')

# plt.scatter(new_points[:, 0], new_points[:, 1], color='brown', label="New Points")

plt.legend()
plt.title("Original and Generated Points in 2D t-SNE Space")
plt.show()


# Apply train model on new points

In [26]:
new_points_test = torch.tensor(new_points).float()
outputs_new_points = inverse_model(new_points_test)
outputs_new_points =outputs_new_points.detach().numpy()

## Visualize

In [27]:
%matplotlib qt

# Create a figure and 3D axis
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(projection='3d')

# # Define colors for each Gaussian distribution
# colors = ['r', 'g', 'b']  # Red, Green, Blue

# Loop through each Gaussian to plot points with corresponding color
for i in range(n_gauss):
    ax.scatter(D[c == i, 0], D[c == i, 1], D[c == i, 2], color=colors[i], alpha=0.7, label=f'Gaussian {i+1}')
    ax.scatter(outputs_new_points[new_labels == i, 0], outputs_new_points[new_labels == i, 1], outputs_new_points[new_labels == i, 2], marker=markers[i],alpha=1.0, s=150, edgecolors='black', label=f'New_points_Gaussian {i+1}')

# ax.scatter(outputs_new_points[:, 0], outputs_new_points[:, 1], outputs_new_points[:, 2], color='k', label=f'Predicted_Gaussian {i+1}')

# Set labels and title
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
ax.set_title(' New points (2D TSNE) mapping into 3D Gaussian Distributions')

# Add a legend
ax.legend()

# Show the plot
plt.show()

## Define Jacobian

In [28]:
# Function to compute Jacobian at a specific point
# def compute_jacobian(x, y, eps):
#     # eps = 1e-5  # Small epsilon for numerical differentiation

#     # Partial derivatives with respect to x 
#     point_hor_plus = torch.tensor([[x + eps, y]]) 
#     point_hor_minus = torch.tensor([[x - eps, y]]) 
#     f_x_plus_eps = inverse_model(point_hor_plus).detach().numpy()   #3D output
#     f_x_minus_eps = inverse_model(point_hor_minus).detach().numpy()
#     df_dx = (f_x_plus_eps - f_x_minus_eps) / (2 * eps)

#     # Partial derivatives with respect to y
#     point_ver_plus = torch.tensor([[x , y + eps]]) 
#     point_ver_minus = torch.tensor([[x , y - eps]]) 
#     f_y_plus_eps = inverse_model(point_ver_plus).detach().numpy()
#     f_y_minus_eps = inverse_model(point_ver_minus).detach().numpy()
#     df_dy = (f_y_plus_eps - f_y_minus_eps) / (2 * eps)

#     # Jacobian matrix 3x2
#     J = np.column_stack((df_dx.T, df_dy.T))
#     return J

import torch
import numpy as np


def compute_jacobian(grid_points):
    """
    Computes the Jacobian matrix for each point in the grid.
    
    Args:
        grid_points (ndarray): A 2D array of shape (n_points, 2) representing the grid points.

    Returns:
        jacobian_matrices (list): A list of jacobian matrices for each grid point.
    """
    jacobian_matrices = []
    
    # Define the model's forward pass to use autograd
    def model_forward(input):
        return inverse_model(input)  # Model's forward pass
    
        # Iterate through the grid points
    for point in grid_points:
        point_tensor = torch.tensor([point], dtype=torch.float32, requires_grad=True)  # (1, 2) tensor
        
        # Compute the Jacobian using autograd's jacobian function
        jacobian = torch.autograd.functional.jacobian(model_forward, point_tensor)
        
        # The output of jacobian will have shape (1, 3, 2), so we need to squeeze to get (3, 2)
        jacobian_matrices.append(jacobian.squeeze(0))  # Remove the batch dimension
    
    return jacobian_matrices

def compute_jacobian_implement(x, y, eps=1e-5):
    # Create tensor point for cloning
    point = torch.tensor([[x, y]], dtype=torch.float32)

    # Partial derivative w.r.t. x using five-point stencil
    f_x_2plus = inverse_model(torch.tensor([[x + 2 * eps, y]], dtype=torch.float32))
    f_x_plus = inverse_model(torch.tensor([[x + eps, y]], dtype=torch.float32))
    f_x_minus = inverse_model(torch.tensor([[x - eps, y]], dtype=torch.float32))
    f_x_2minus = inverse_model(torch.tensor([[x - 2 * eps, y]], dtype=torch.float32))
    
    df_dx = (-f_x_2plus + 8 * f_x_plus - 8 * f_x_minus + f_x_2minus) / (12 * eps)

    # Partial derivative w.r.t. y using five-point stencil
    f_y_2plus = inverse_model(torch.tensor([[x, y + 2 * eps]], dtype=torch.float32))
    f_y_plus = inverse_model(torch.tensor([[x, y + eps]], dtype=torch.float32))
    f_y_minus = inverse_model(torch.tensor([[x, y - eps]], dtype=torch.float32))
    f_y_2minus = inverse_model(torch.tensor([[x, y - 2 * eps]], dtype=torch.float32))
    
    df_dy = (-f_y_2plus + 8 * f_y_plus - 8 * f_y_minus + f_y_2minus) / (12 * eps)

    # Stack results to form Jacobian matrix
    jacobian = torch.stack([df_dx.squeeze(), df_dy.squeeze()], dim=1)
    
    return jacobian.detach().numpy()

In [29]:
def compute_jacobian(x, y):
    """
    Computes the Jacobian matrix for each point in the grid.
    
    Args:
        grid_points (ndarray): A 2D array of shape (n_points, 2) representing the grid points.

    Returns:
        jacobian_matrices (list): A list of jacobian matrices for each grid point.
    """
    jacobian_matrices = []
    
    # Define the model's forward pass to use autograd
    def model_forward(input):
        return inverse_model(input)  # Model's forward pass
    
    # Iterate through the grid points
    # for point in grid_points:
    point_tensor = torch.tensor([x, y], dtype=torch.float32, requires_grad=True)  # (1, 2) tensor
    
    # Compute the Jacobian using autograd's jacobian function
    jacobian = torch.autograd.functional.jacobian(model_forward, point_tensor)
    
        # The output of jacobian will have shape (1, 3, 2), so we need to squeeze to get (3, 2)
        # jacobian_matrices.append(jacobian.squeeze(0))  # Remove the batch dimension
    
    return jacobian

In [30]:
# Calculate Jacobians over the grid and store results
jacobians = []
for i in range(num_grid_points):
    for j in range(num_grid_points):
        x, y = xx[i, j], yy[i, j]
        print(x,y)
        jacobian = compute_jacobian(x, y)
        # jacobian = compute_jacobian_implement(x, y, 1e-5)
        # print(jacobian)
        jacobians.append(jacobian)

-3.6020608 -7.361164
-3.3995872 -7.361164
-3.1971133 -7.361164
-2.9946396 -7.361164
-2.7921658 -7.361164
-2.589692 -7.361164
-2.3872185 -7.361164
-2.1847448 -7.361164
-1.982271 -7.361164
-1.7797972 -7.361164
-1.5773234 -7.361164
-1.3748498 -7.361164
-1.1723762 -7.361164
-0.9699023 -7.361164
-0.76742864 -7.361164
-0.56495476 -7.361164
-0.36248112 -7.361164
-0.16000748 -7.361164
0.042466402 -7.361164
0.24494004 -7.361164
0.44741392 -7.361164
0.64988756 -7.361164
0.8523612 -7.361164
1.0548348 -7.361164
1.2573085 -7.361164
1.4597826 -7.361164
1.6622562 -7.361164
1.8647299 -7.361164
2.0672035 -7.361164
2.2696772 -7.361164
2.4721513 -7.361164
2.674625 -7.361164
2.8770986 -7.361164
3.0795722 -7.361164
3.2820458 -7.361164
3.48452 -7.361164
3.6869936 -7.361164
3.8894672 -7.361164
4.091941 -7.361164
4.2944145 -7.361164
4.4968886 -7.361164
4.6993623 -7.361164
4.901836 -7.361164
5.1043096 -7.361164
5.306783 -7.361164
5.509257 -7.361164
5.7117305 -7.361164
5.914204 -7.361164
6.1166778 -7.361164
6.3

In [31]:
print(jacobians[0].shape)
print(len(jacobians))

jacobians

torch.Size([3, 2])
10000


[tensor([[-0.0157,  0.0430],
         [ 0.0115,  0.0148],
         [-0.0192,  0.0179]]),
 tensor([[-0.0181,  0.0436],
         [ 0.0130,  0.0141],
         [-0.0214,  0.0181]]),
 tensor([[-0.0177,  0.0425],
         [ 0.0168,  0.0121],
         [-0.0212,  0.0177]]),
 tensor([[-0.0188,  0.0431],
         [ 0.0160,  0.0129],
         [-0.0221,  0.0176]]),
 tensor([[-0.0178,  0.0426],
         [ 0.0170,  0.0128],
         [-0.0220,  0.0174]]),
 tensor([[-0.0158,  0.0406],
         [ 0.0189,  0.0118],
         [-0.0220,  0.0164]]),
 tensor([[-0.0154,  0.0399],
         [ 0.0214,  0.0112],
         [-0.0240,  0.0165]]),
 tensor([[-0.0163,  0.0392],
         [ 0.0225,  0.0114],
         [-0.0264,  0.0159]]),
 tensor([[-0.0164,  0.0387],
         [ 0.0239,  0.0111],
         [-0.0267,  0.0159]]),
 tensor([[-0.0156,  0.0381],
         [ 0.0252,  0.0112],
         [-0.0281,  0.0158]]),
 tensor([[-0.0157,  0.0377],
         [ 0.0266,  0.0116],
         [-0.0327,  0.0156]]),
 tensor([[-0.0159,  0

## Reshaping Jacobian [num_girds, num_grids, 3, 2]

In [32]:

# Convert the list of numpy arrays into a list of PyTorch tensors
jacobian_tensors = [torch.tensor(jacob) for jacob in jacobians]

# Convert the list into a 3D tensor
jacobian_tensor = torch.stack(jacobian_tensors)  # Shape will be [num_grids * num_grids, 3, 2]

# Reshape the tensor to [num_grids, num_grids, 3, 2]
jacobian_tensor_reshaped = jacobian_tensor.view(num_grid_points, num_grid_points, 3, 2)
jacobian_tensor_reshaped.shape

C:\Users\waqar\AppData\Local\Temp\ipykernel_25584\3012617843.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  jacobian_tensors = [torch.tensor(jacob) for jacob in jacobians]


torch.Size([100, 100, 3, 2])

In [33]:
# import torch
# import matplotlib.pyplot as plt
# import numpy as np

# # Prepare a combined heatmap data
# # Each Jacobian matrix has a shape of (3, 2)
# combined_heatmap = torch.zeros((num_grid_points * 3, num_grid_points * 2))  # Create an empty array for combined heatmap

# # Fill in the combined heatmap
# for i in range(num_grid_points):
#     for j in range(num_grid_points):
#         jacobian_matrix = jacobian_tensor_reshaped[i, j]
        
#         # Fill the appropriate section in the combined heatmap with the Jacobian matrix
#         combined_heatmap[i * 3:(i + 1) * 3, j * 2:(j + 1) * 2] = jacobian_matrix

# # Plot the combined heatmap
# plt.figure(figsize=(10, 10))
# plt.imshow(combined_heatmap, aspect='auto', cmap='viridis', interpolation='nearest')
# plt.title('Combined Jacobian Matrices Heatmap')
# plt.colorbar(label='Value')
# plt.xlabel('Grid Points (2D space)')
# plt.ylabel('Jacobian Matrix Values (3D)')
# # plt.xticks(np.arange(0, num_grid_points * 2, 2), np.arange(num_grid_points))  # X ticks for grid points
# # plt.yticks(np.arange(0, num_grid_points * 3, 3), np.arange(num_grid_points))  # Y ticks for Jacobian matrix rows

# # Show the heatmap
# plt.tight_layout()
# plt.show()


In [34]:
# import torch
# import matplotlib.pyplot as plt
# import numpy as np
# from matplotlib.colors import ListedColormap

# # Prepare a combined heatmap data
# combined_heatmap = torch.zeros((num_grid_points * 3, num_grid_points * 2))  # Create an empty array for combined heatmap

# # Fill in the combined heatmap
# for i in range(num_grid_points):
#     for j in range(num_grid_points):
#         jacobian_matrix = jacobian_tensor_reshaped[i, j]
        
#         # Fill the appropriate section in the combined heatmap with the Jacobian matrix
#         combined_heatmap[i * 3:(i + 1) * 3, j * 2:(j + 1) * 2] = jacobian_matrix

# # Define a custom colormap with three colors
# colors = ['blue', 'green', 'red']  # Adjust these colors as needed
# cmap = ListedColormap(colors)

# # Normalize the data to fit into the three categories
# # Example: Normalize values to three ranges (low, medium, high)
# combined_heatmap_norm = torch.zeros_like(combined_heatmap)

# # Assuming the values range from a min to a max, you can categorize them
# low_threshold = combined_heatmap.min() + (combined_heatmap.max() - combined_heatmap.min()) / 3
# high_threshold = combined_heatmap.min() + 2 * (combined_heatmap.max() - combined_heatmap.min()) / 3

# for i in range(combined_heatmap.shape[0]):
#     for j in range(combined_heatmap.shape[1]):
#         if combined_heatmap[i, j] < low_threshold:
#             combined_heatmap_norm[i, j] = 0  # Low values
#         elif combined_heatmap[i, j] < high_threshold:
#             combined_heatmap_norm[i, j] = 1  # Medium values
#         else:
#             combined_heatmap_norm[i, j] = 2  # High values

# # Plot the combined heatmap with three colors
# plt.figure(figsize=(10, 10))
# plt.imshow(combined_heatmap_norm, aspect='auto', cmap=cmap, interpolation='nearest')
# plt.title('Combined Jacobian Matrices Heatmap (3 Colors)')
# plt.colorbar(label='Value (Categorized)', ticks=[0, 1, 2], format='%d')
# plt.xlabel('Grid Points (2D space)')
# plt.ylabel('Jacobian Matrix Values (3D)')
# plt.xticks(np.arange(0, num_grid_points * 2, 2), np.arange(num_grid_points))  # X ticks for grid points
# plt.yticks(np.arange(0, num_grid_points * 3, 3), np.arange(num_grid_points))  # Y ticks for Jacobian matrix rows

# # Show the heatmap
# plt.tight_layout()
# plt.show()


#  Contours

In [35]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate the Jacobian norm at each grid point
jacobian_norms = torch.linalg.matrix_norm(jacobian_tensor_reshaped, dim=(2, 3)).numpy()  #Shape [num_grids, num_grids]

# Inspect the distribution of norms to decide thresholds
mean_norm = np.mean(jacobian_norms)
std_norm = np.std(jacobian_norms)
print(f"Mean: {mean_norm}, Std Dev: {std_norm}")

# Define threshold levels based on mean and standard deviation
# Use levels that are one standard deviation below and above the mean
levels = [mean_norm - std_norm, mean_norm, mean_norm + std_norm]

# Plotting
plt.figure(figsize=(8, 6))
contour = plt.contourf(jacobian_norms, levels=levels, cmap="coolwarm", extend='both')
plt.colorbar(contour, label="Jacobian Norm")
plt.title("Contour Plot with Threshold Levels")
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.show()


Mean: 0.10419707000255585, Std Dev: 0.052261825650930405


In [36]:
jacobian_norms.shape

(100, 100)

In [37]:
# import torch
# import numpy as np
# import matplotlib.pyplot as plt

# jacobian_norms = torch.norm(jacobian_tensor_reshaped, dim=(2, 3)).numpy()

# # Use quantiles for levels
# q25, q50, q75 = np.percentile(jacobian_norms, [25, 50, 75])
# levels_quantiles = [q25, q50, q75]  # You can add more levels if needed

# # Normalize jacobian norms (values between 0 and 1)
# jacobian_norms_normalized = (jacobian_norms - np.min(jacobian_norms)) / (np.max(jacobian_norms) - np.min(jacobian_norms))

# # # Set custom levels based on data range (or normalized data range)
# # min_val, max_val = np.min(jacobian_norms), np.max(jacobian_norms)
# # # min_val, max_val = np.min(jacobian_norms_normalized), np.max(jacobian_norms_normalized)
# # levels_custom = np.linspace(min_val, max_val, num=5)  # Generates 5 evenly spaced levels

# # Plot using quantile-based levels
# plt.figure(figsize=(8, 6))
# contour = plt.contourf(jacobian_norms, levels=levels_quantiles, cmap="coolwarm", extend='both')
# # contour = plt.contourf(jacobian_norms, levels=levels_custom, cmap="coolwarm", extend='both')
# plt.colorbar(contour, label="Jacobian Norm")
# plt.title("Contour Plot with Quantile-Based Levels")
# plt.xlabel("X-axis")
# plt.ylabel("Y-axis")
# plt.show()


## Jacobian norm Heatmap

In [38]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming jacobian_tensor has shape [num_grid_points, num_grid_points, 3, 2]
print("Shape of jacobian_tensor:", jacobian_tensor_reshaped.shape)  # Verify shape

# Calculate the Jacobian norm at each grid point (sum of squared elements in each 3x2 matrix)
jacobian_norms = torch.linalg.matrix_norm(jacobian_tensor_reshaped, dim=(2, 3)).numpy()  # shape [num_grid_points, num_grid_points]
jacobian_norms.shape
# Step 2: Plot the Jacobian norm heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(jacobian_norms, xticklabels=False, yticklabels=False, cmap="coolwarm", cbar_kws={'label': 'Jacobian Norm'})

# # Flip the data matrix vertically to align with Cartesian
# flipped_data = np.flipud(jacobian_norms)

# # Plot heatmap
# sns.heatmap(flipped_data, xticklabels=x_vals, yticklabels=np.flip(y_vals), cmap='hot')
# plt.gca().invert_yaxis()  # Align y-axis to Cartesian

plt.title("Jacobian Norm Heatmap - Approximate Decision Boundaries")
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.show()


Shape of jacobian_tensor: torch.Size([100, 100, 3, 2])


# Spectral norm

In [39]:
# import torch
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Calculate spectral norm (largest singular value) for each Jacobian matrix in the grid
# spectral_norms = np.zeros((num_grid_points, num_grid_points))  # Placeholder for storing spectral norms

# for i in range(num_grid_points):
#     for j in range(num_grid_points):
#         jacobian_matrix = jacobian_tensor_reshaped[i, j]  # Shape [3, 2]
#         singular_values = torch.linalg.svdvals(jacobian_matrix)  # Get singular values
#         # print(singular_values)
#         spectral_norms[i, j] = singular_values.max().item()  # Largest singular value (spectral norm)

# # Step 2: Plot the heatmap
# plt.figure(figsize=(8, 6))
# sns.heatmap(spectral_norms, annot=False, cmap="hot", cbar=True)
# plt.title("Spectral Norm Heatmap of Jacobians")
# plt.xlabel("X-axis")
# plt.ylabel("Y-axis")
# # plt.colorbar(label="Spectral Norm")
# plt.show()


# SVD on Jacobian

In [40]:
# Initialize lists to store singular values, U and V matrices for each Jacobian
singular_values_list = []
U_matrices = []
V_matrices = []

# Iterate over each grid point and apply SVD
for i in range(jacobian_tensor_reshaped.shape[0]):
    for j in range(jacobian_tensor_reshaped.shape[1]):
        jacobian_matrix = jacobian_tensor_reshaped[i, j]  # Shape: (3, 2)
        
        
        # Perform SVD
        U, SV, Vt = torch.linalg.svd(jacobian_matrix, full_matrices=False)
        # print(SV)
        # np.savetxt('jacob_matrix'+ str(i),jacobian_matrix)
    
        # break
        # Store the singular values and U, V matrices
        singular_values_list.append(SV)
        U_matrices.append(U)
        V_matrices.append(Vt)

In [41]:
print(U_matrices[0].shape)
print(singular_values_list[0].shape)
print(V_matrices[0].shape)

print(len(U_matrices))
print(len(singular_values_list))
print(len(V_matrices))

torch.Size([3, 2])
torch.Size([2])
torch.Size([2, 2])
10000
10000
10000


In [42]:
# Define min and max values
x_min, x_max = np.min(S[:, 0]), np.max(S[:, 0])
y_min, y_max = np.min(S[:, 1]), np.max(S[:, 1])
print(x_min, x_max)
print(y_min, y_max)

# Reshape singular values to the grid's shape for visualization
# Extract the largest singular value for each point in the grid
largest_singular_values = np.array([s[0] for s in singular_values_list]).reshape(num_grid_points, num_grid_points)


# Plot heatmap
plt.figure(figsize=(8, 6))
plt.imshow(largest_singular_values,
           extent=(x_min, x_max, y_min, y_max),
           origin='lower',
           cmap='hot', 
           interpolation='nearest')
plt.colorbar(label="Largest Singular Value")
plt.xlabel("Grid X")
plt.ylabel("Grid Y")
plt.title("Heatmap of Jacobian Singular Values")
plt.show()


-3.6020608 16.442839
-7.361164 18.883371


# Eignevalue analysis

In [43]:
import numpy as np
import torch
import matplotlib.pyplot as plt


# Compute the eigenvalues of J^T J for each Jacobian matrix
eigenvalues = np.zeros((num_grid_points, num_grid_points, 2))  # Store two eigenvalues per J^T J (2x2 matrix)

for i in range(num_grid_points):
    for j in range(num_grid_points):
        jacobian = jacobian_tensor_reshaped[i, j]  # Get the 3x2 Jacobian matrix
        gram_matrix = jacobian.T @ jacobian  # Compute the 2x2 J^T J matrix
        eigvals = torch.linalg.eigvals(gram_matrix)  # Eigenvalues of J^T J
        eigenvalues[i, j] = eigvals.real.numpy()  # Store only real parts if complex

# Visualize the largest eigenvalue across the grid to identify sensitive areas
largest_eigenvalue = np.max(eigenvalues, axis=2)
# sqrt_large_eig_val = np.sqrt(largest_eigenvalue)

plt.figure(figsize=(8, 6))
plt.imshow(largest_eigenvalue,
           extent=(x_min, x_max, y_min, y_max),
           origin='lower', 
           cmap="hot")
# plt.imshow(sqrt_large_eig_val, cmap="hot")
plt.colorbar(label="Largest Eigenvalue of J^T J (Sensitivity)")
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.title("Heatmap of Largest Eigenvalue (Sensitive Regions)")
plt.show()


# Overlay plot

In [44]:


# Define grid resolution 
num_grid_points = 100

# Generate grid
x_vals = np.linspace(x_min, x_max, num_grid_points)
y_vals = np.linspace(y_min, y_max, num_grid_points)
xx, yy = np.meshgrid(x_vals, y_vals)
grid_points = np.c_[xx.ravel(), yy.ravel()]



jacobian_norms = np.zeros(len(grid_points))
for idx, point in enumerate(grid_points):
    point_tensor = torch.tensor(point, dtype=torch.float32, requires_grad=True).view(1, 2)
    
    # Compute the Jacobian for the current point
    jacobian = torch.autograd.functional.jacobian(lambda x: inverse_model(x), point_tensor)
    
    # Reshape Jacobian to 2D: (output_dim, input_dim)
    jacobian_2d = jacobian.view(3, 2)  # Assuming output is (1, 3), input is (1, 2)
    
    # Compute spectral norm (largest singular value)
    jacobian_norms[idx] = torch.linalg.norm(jacobian_2d, ord=2).item()

jacobian_norms = jacobian_norms.reshape(xx.shape)

# Step 4: Plot heatmap with t-SNE points overlayed
plt.figure(figsize=(10, 8))

# Overlay t-SNE points
# plt.scatter(S[:, 0], S[:, 1], c='blue', edgecolor='k', label='t-SNE points')
for i in range(n_gauss):
    plt.scatter(S[c == i, 0], S[c == i, 1], color=colors[i], label=f'Gaussian{i+1}', edgecolor=None)

# Plot heatmap
plt.imshow(
    jacobian_norms,
    extent=(x_min, x_max, y_min, y_max),
    origin='lower',
    cmap='hot',
    alpha=1
)
plt.colorbar(label='Spectral Norm of Jacobian')




# Labels and title
plt.title("Overlaying t-SNE points on Jacobian Heatmap")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
# plt.legend()
plt.show()


# # Step 4: Plot the heatmap and overlay t-SNE points
# plt.figure(figsize=(10, 8))
# plt.imshow(largest_eigenvalue, extent=(x_min, x_max, y_min, y_max), origin='lower', cmap='hot', alpha=0.6)
# # plt.imshow(largest_eigenvalue, extent=(x_min, x_max, y_min, y_max), origin='lower', cmap='hot', alpha=0.6)
# plt.colorbar(label='Spectral Norm of Jacobian')
# plt.scatter(S[:, 0], S[:, 1], c=c, cmap='viridis', edgecolor='k', label='t-SNE points')
# plt.title("Overlaying t-SNE points on Jacobian Heatmap")
# plt.xlabel("t-SNE Dimension 1")
# plt.ylabel("t-SNE Dimension 2")
# plt.legend()
# plt.show()

# Quiver plot

In [64]:
import numpy as np
import torch
import matplotlib.pyplot as plt


# Initialize grid for plotting
x, y = np.meshgrid(np.arange(num_grid_points), np.arange(num_grid_points))

# Prepare u and v by normalizing the Jacobian vectors
u = np.zeros((num_grid_points, num_grid_points))
v = np.zeros((num_grid_points, num_grid_points))

# Normalize each Jacobian's first row component
for i in range(num_grid_points):
    for j in range(num_grid_points):
        jacobian = jacobian_tensor_reshaped[i, j]  # 3x2 matrix
        vector = jacobian[0, :2].numpy()  # Get the first row as the vector (u, v)
        norm = np.linalg.norm(vector)
        if norm > 0:
            u[i, j], v[i, j] = vector / norm  # Normalize to unit length

# Plotting with normalized vectors
plt.figure(figsize=(8, 6))
plt.quiver(x, y, u, v, color='blue', angles='xy', scale_units='xy', scale=1)  # scale=1 for normalized
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.title("Quiver Plot of Jacobian Directions")
plt.grid()
plt.show()


In [46]:
# import numpy as np
# import torch
# import matplotlib.pyplot as plt

# # Example Jacobian tensor
# # jacobian_tensor = torch.rand((10, 10, 3, 2))  # Adjust this with your actual data

# # Initialize grid for plotting
# x, y = np.meshgrid(np.arange(num_grid_points), np.arange(num_grid_points))

# # Prepare u and v by normalizing the Jacobian vectors
# u = np.zeros((num_grid_points, num_grid_points))
# v = np.zeros((num_grid_points, num_grid_points))
# magnitudes = np.zeros((num_grid_points, num_grid_points))

# # Normalize each Jacobian's first row component
# for i in range(num_grid_points):
#     for j in range(num_grid_points):
#         jacobian = jacobian_tensor_reshaped[i, j]  # 3x2 matrix
#         print(jacobian)
#         vector = jacobian[0, :2].numpy()  # Get the first row as the vector (u, v)
#         print(vector)
#         norm = np.linalg.norm(vector)
#         print(norm)
#         if norm > 0:
#             u[i, j], v[i, j] = vector / norm  # Normalize to unit length
#             # u[i, j], v[i, j] = vector   # Normalize to unit length
#             magnitudes[i, j] = norm

# vmin, vmax = 0, np.max(magnitudes) 
# print(vmin, vmax) 
# # Create quiver plot with color mapped to vector magnitude
# plt.figure(figsize=(8, 6))
# quiver = plt.quiver(x, y, u, v, magnitudes, cmap='plasma', angles='xy', scale_units='xy', scale=1)
# plt.colorbar(quiver, label="Magnitude of Vectors")
# plt.xlabel("X-axis")
# plt.ylabel("Y-axis")
# plt.title("Quiver Plot with Magnitude-based Color Mapping")
# plt.grid()
# plt.show()


# Visualization of Singular Vectors

In [47]:
# Convert the list into a 3D tensor
U_mat_tensors = torch.stack(U_matrices)  # Shape will be [num_grids * num_grids, 3, 2]
# Reshape the tensor to [num_grids, num_grids, 3, 2]
U_mat_tensors_reshaped = U_mat_tensors.view(num_grid_points, num_grid_points, 3, 2)
print(U_mat_tensors_reshaped.shape)

# Convert the list into a 3D tensor
V_mat_tensors = torch.stack(V_matrices)  # Shape will be [num_grids * num_grids, 2, 2]
V_mat_tensors.shape
# Reshape the tensor to [num_grids, num_grids, 3, 2]
V_mat_tensors_reshaped = V_mat_tensors.view(num_grid_points, num_grid_points, 2, 2)
print(V_mat_tensors_reshaped.shape)

torch.Size([100, 100, 3, 2])
torch.Size([100, 100, 2, 2])


In [48]:
U_matrices =U_mat_tensors_reshaped
V_matrices = V_mat_tensors_reshaped

for i in range(num_grid_points):
    for j in range(num_grid_points):
        jacobian = jacobian_tensor_reshaped[i, j]
        u, s, v = torch.svd(jacobian)
        U_matrices[i, j] = u
        V_matrices[i, j] = v

# Visualize each component of U and V to examine directional changes
fig, axes = plt.subplots(2, 3, figsize=(12, 10))
axes = axes.ravel()

for k in range(3):  # Components of U
    im = axes[k].imshow(U_matrices[:, :, k, 0], cmap="viridis")
    fig.colorbar(im, ax=axes[k])
    axes[k].set_title(f"Component {k+1} of U")

for k in range(2):  # Components of V
    im = axes[3 + k].imshow(V_matrices[:, :, k, 0], cmap="viridis")
    fig.colorbar(im, ax=axes[3 + k])
    axes[3 + k].set_title(f"Component {k+1} of V")

# Turn off the last subplot
axes[5].axis('off')  # Make the 6th subplot invisible
plt.tight_layout()
plt.show()


# Visualize determinant of Jacobian

In [49]:
# Compute the determinant of each Jacobian
determinants = np.zeros((num_grid_points, num_grid_points))

for i in range(num_grid_points):
    for j in range(num_grid_points):
        jacobian = jacobian_tensor_reshaped[i, j]
        determinants[i, j] = torch.det(jacobian @ jacobian.T).sqrt().item()  # Determinant of J @ J^T for stability

# Compute the gradient of the determinants
gradient_determinants = np.gradient(determinants)

# Visualize the absolute gradient as a heatmap
gradient_magnitude = np.sqrt(gradient_determinants[0]**2 + gradient_determinants[1]**2)

plt.figure(figsize=(8, 6))
plt.imshow(gradient_magnitude, cmap="hot")
plt.colorbar(label="Gradient Magnitude of Determinants")
plt.title("Heatmap of Determinant Gradient (Potential Boundaries ?)")
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.show()


# Orthognal Plan

In [50]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Placeholder for U_matrices (typically obtained from SVD)
U_matrices = U_mat_tensors_reshaped

# Placeholder for orthogonal vectors (W and W')
W_matrices = torch.zeros(num_grid_points, num_grid_points, 3)
W_prime_matrices = torch.zeros(num_grid_points, num_grid_points, 3)

# Calculate W and W' for each grid point
for i in range(num_grid_points):
    for j in range(num_grid_points):
        U1 = U_matrices[i, j, :, 0]  # First column of U matrix (primary direction)
        # V = torch.tensor([1, 0, 0]) if U1[0] == 0 else torch.tensor([0, 1, 0])  # Arbitrary vector
        V = torch.tensor([1.0, 0.0, 0.0]) if U1[0] == 0 else torch.tensor([0.0, 1.0, 0.0])


        W = torch.cross(U1, V)  # First orthogonal vector
        W_prime = torch.cross(U1, W)  # Second orthogonal vector

        W_matrices[i, j] = W
        W_prime_matrices[i, j] = W_prime

# Visualization setup
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Select a few representative grid points for visualization (e.g., corners and center)
sample_points = [(0, 0), (0, num_grid_points-1), (num_grid_points-1, 0), (num_grid_points-1, num_grid_points-1), (num_grid_points//2, num_grid_points//2)]

for i, j in sample_points:
    # Origin for each plane
    origin = np.array([i, j, 0])

    # Primary direction vector (U1)
    U1 = U_matrices[i, j, :, 0].numpy()
    ax.quiver(*origin, *U1, color='r', length=1.0, normalize=True, label="U1 (Primary)" if (i, j) == sample_points[0] else "")

    # Orthogonal vectors spanning the plane (W and W')
    W = W_matrices[i, j].numpy()
    W_prime = W_prime_matrices[i, j].numpy()
    ax.quiver(*origin, *W, color='b', length=1.0, normalize=True, label="W (Plane Basis)" if (i, j) == sample_points[0] else "")
    ax.quiver(*origin, *W_prime, color='g', length=1.0, normalize=True, label="W' (Plane Basis)" if (i, j) == sample_points[0] else "")

# Setting plot limits and labels
ax.set_xlim([0, num_grid_points])
ax.set_ylim([0, num_grid_points])
ax.set_zlim([-1, 1])
ax.set_xlabel("Grid X")
ax.set_ylabel("Grid Y")
ax.set_zlabel("Z")

# Adding legend
ax.legend(loc='upper left')
plt.title("Orthogonal Planes at Selected Grid Points")
plt.show()


C:\Users\waqar\AppData\Local\Temp\ipykernel_25584\3553001366.py:21: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\Cross.cpp:66.)
  W = torch.cross(U1, V)  # First orthogonal vector


In [51]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Example dimensions for the Jacobian grid
num_grid_points, num_grid_points = 10, 10  # Modify as needed

# Placeholder for U_matrices (typically obtained from SVD)
U_matrices = U_mat_tensors_reshaped

# Placeholder for orthogonal vectors (W and W')
W_matrices = torch.zeros(num_grid_points, num_grid_points, 3)
W_prime_matrices = torch.zeros(num_grid_points, num_grid_points, 3)

# Calculate W and W' for each grid point
for i in range(num_grid_points):
    for j in range(num_grid_points):
        U1 = U_matrices[i, j, :, 0]  # First column of U matrix (primary direction)
        V = torch.tensor([1.0, 0.0, 0.0]) if U1[0] == 0 else torch.tensor([0.0, 1.0, 0.0])  # Arbitrary vector

        W = torch.cross(U1, V)  # First orthogonal vector
        W_prime = torch.cross(U1, W)  # Second orthogonal vector

        W_matrices[i, j] = W
        W_prime_matrices[i, j] = W_prime

# Visualization setup
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Scale for the vectors to avoid clutter
vector_scale = 0.3

# Iterate over all grid points to plot the planes
for i in range(num_grid_points):
    for j in range(num_grid_points):
        # Origin for each plane
        origin = np.array([i, j, 0])

        # Primary direction vector (U1)
        U1 = U_matrices[i, j, :, 0].numpy() * vector_scale
        ax.quiver(*origin, *U1, color='r', alpha=0.5, length=1.0, normalize=True)

        # Orthogonal vectors spanning the plane (W and W')
        W = W_matrices[i, j].numpy() * vector_scale
        W_prime = W_prime_matrices[i, j].numpy() * vector_scale
        ax.quiver(*origin, *W, color='b', alpha=0.5, length=1.0, normalize=True)
        ax.quiver(*origin, *W_prime, color='g', alpha=0.5, length=1.0, normalize=True)

# Setting plot limits and labels
ax.set_xlim([0, num_grid_points])
ax.set_ylim([0, num_grid_points])
ax.set_zlim([-1, 1])
ax.set_xlabel("Grid X")
ax.set_ylabel("Grid Y")
ax.set_zlabel("Z")

plt.title("Orthogonal Planes at All Grid Points")
plt.show()


# Solid Plane

In [52]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Define grid dimensions
num_grid_points, num_grid_points = 10, 10  # Adjust to a smaller grid for clear visualization

# Placeholder for U_matrices (typically obtained from SVD)
# U_matrices = torch.randn(num_grid_points, num_grid_points, 3, 3)  # Random U matrices

# Placeholder for orthogonal vectors (W and W')
W_matrices = torch.zeros(num_grid_points, num_grid_points, 3)
W_prime_matrices = torch.zeros(num_grid_points, num_grid_points, 3)

# Compute orthogonal vectors for each grid point
for i in range(num_grid_points):
    for j in range(num_grid_points):
        U1 = U_matrices[i, j, :, 0]  # First column of U matrix (primary direction)
        V = torch.tensor([1.0, 0.0, 0.0]) if U1[0] == 0 else torch.tensor([0.0, 1.0, 0.0])  # Arbitrary vector

        W = torch.cross(U1, V)  # First orthogonal vector
        W_prime = torch.cross(U1, W)  # Second orthogonal vector

        W_matrices[i, j] = W
        W_prime_matrices[i, j] = W_prime

# Visualization setup
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Scale factor for the plane's extent
plane_scale = 0.9

# Iterate over each grid point to plot the planes
for i in range(num_grid_points):
    for j in range(num_grid_points):
        # Origin for each plane
        origin = np.array([i, j, 0])

        # Basis vectors for the plane (scaled for visibility)
        W = W_matrices[i, j].numpy() * plane_scale
        W_prime = W_prime_matrices[i, j].numpy() * plane_scale

        # Create a meshgrid for the plane based on W and W_prime
        s, t = np.meshgrid(np.linspace(-1, 1, 5), np.linspace(-1, 1, 5))
        plane_x = origin[0] + s * W[0] + t * W_prime[0]
        plane_y = origin[1] + s * W[1] + t * W_prime[1]
        plane_z = origin[2] + s * W[2] + t * W_prime[2]

        # Plot the plane as a surface
        ax.plot_surface(plane_x, plane_y, plane_z, color="cyan", alpha=0.5, edgecolor="none")

# Setting plot limits and labels
ax.set_xlim([0, num_grid_points])
ax.set_ylim([0, num_grid_points])
ax.set_zlim([-1, 1])
ax.set_xlabel("Grid X")
ax.set_ylabel("Grid Y")
ax.set_zlabel("Z")
plt.title("Solid Planes at Grid Points")
plt.show()


In [53]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import Patch

# Define grid dimensions
num_grid_points, num_grid_points = 100, 100  # Adjust to a smaller grid for clear visualization

# Placeholder for U_matrices (typically obtained from SVD)
# U_matrices = torch.randn(num_grid_points, num_grid_points, 3, 3)  # Random U matrices

# Placeholder for orthogonal vectors (W and W')
W_matrices = torch.zeros(num_grid_points, num_grid_points, 3)
W_prime_matrices = torch.zeros(num_grid_points, num_grid_points, 3)

# Compute orthogonal vectors for each grid point
for i in range(num_grid_points):
    for j in range(num_grid_points):
        U1 = U_matrices[i, j, :, 0]  # First column of U matrix (primary direction)
        V = torch.tensor([1.0, 0.0, 0.0]) if U1[0] == 0 else torch.tensor([0.0, 1.0, 0.0])  # Arbitrary vector

        W = torch.cross(U1, V)  # First orthogonal vector
        W_prime = torch.cross(U1, W)  # Second orthogonal vector

        W_matrices[i, j] = W
        W_prime_matrices[i, j] = W_prime

# Visualization setup
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Scale factor for the plane's extent
plane_scale = 0.3

# Iterate over each grid point to plot the planes
for i in range(num_grid_points):
    for j in range(num_grid_points):
        # Origin for each plane
        origin = np.array([i, j, 0])

        # Basis vectors for the plane (scaled for visibility)
        W = W_matrices[i, j].numpy() * plane_scale
        W_prime = W_prime_matrices[i, j].numpy() * plane_scale

        # Create a meshgrid for the plane based on W and W_prime
        s, t = np.meshgrid(np.linspace(-1, 1, 5), np.linspace(-1, 1, 5))
        plane_x = origin[0] + s * W[0] + t * W_prime[0]
        plane_y = origin[1] + s * W[1] + t * W_prime[1]
        plane_z = origin[2] + s * W[2] + t * W_prime[2]

        # Plot the plane as a surface
        ax.plot_surface(plane_x, plane_y, plane_z, color="cyan", alpha=0.5, edgecolor="none")

# Setting plot limits and labels
ax.set_xlim([0, num_grid_points])
ax.set_ylim([0, num_grid_points])
ax.set_zlim([-1, 1])
ax.set_xlabel("Grid X")
ax.set_ylabel("Grid Y")
ax.set_zlabel("Z")
plt.title("Solid Planes at Grid Points")

# Create custom legend
legend_elements = [Patch(facecolor='cyan', edgecolor='none', label='Orthogonal Plane')]
ax.legend(handles=legend_elements, loc='upper right')

plt.show()


In [54]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch
import numpy as np

# Assuming you already have U_matrices from SVD
fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(111, projection='3d')

# num_grid_points, num_grid_points = U_matrices.shape[:2]
num_grid_points, num_grid_points = 10,10
plane_size = 0.2  # Size of each plane for visualization

# Iterate over each grid point
for i in range(num_grid_points):
    for j in range(num_grid_points):
        # Get primary direction vector (first column of U)
        U1 = U_matrices[i, j, :, 0]

        # Define two orthogonal vectors
        V = torch.tensor([1, 0, 0], dtype=torch.float32) if U1[0] != 0 else torch.tensor([0, 1, 0], dtype=torch.float32)
        W = torch.cross(U1, V)
        W_prime = torch.cross(U1, W)

        # Define the corners of the rectangular plane
        center = torch.tensor([i, j, 0], dtype=torch.float32)
        corner1 = center - plane_size * (W + W_prime)
        corner2 = center - plane_size * (W - W_prime)
        corner3 = center + plane_size * (W + W_prime)
        corner4 = center + plane_size * (W - W_prime)

        # Create rectangular plane grid
        X_plane = np.array([corner1[0], corner2[0], corner3[0], corner4[0]]).reshape(2, 2)
        Y_plane = np.array([corner1[1], corner2[1], corner3[1], corner4[1]]).reshape(2, 2)
        Z_plane = np.array([corner1[2], corner2[2], corner3[2], corner4[2]]).reshape(2, 2)

        # Plot the plane
        ax.plot_surface(X_plane, Y_plane, Z_plane, color='cyan', alpha=0.3)

        # Add arrow for primary direction vector
        ax.quiver(center[0], center[1], center[2], U1[0], U1[1], U1[2], color='red', length=0.3, normalize=False)

# Axis labels and title
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_title("Orthogonal Planes with Primary Direction Vectors")

# Legend for planes and arrows
plane_patch = plt.Line2D([0], [0], marker='s', color='w', label='Orthogonal Plane', markerfacecolor='cyan', markersize=10, alpha=0.5)
arrow_patch = plt.Line2D([0], [0], marker='|', color='r', label='Primary Direction Vector', markersize=10)
ax.legend(handles=[plane_patch, arrow_patch], loc='upper right')

plt.show()


# 3D Visualizationo of Jacobian Transformation

In [55]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


# Prepare grid
x, y = np.meshgrid(np.arange(num_grid_points), np.arange(num_grid_points))

# Calculate output based on Jacobian (assuming simple linear transformation for demo)
output = np.zeros((num_grid_points, num_grid_points, 3))
for i in range(num_grid_points):
    for j in range(num_grid_points):
        jacobian = jacobian_tensor_reshaped[i, j]  # Shape (3, 2)
        input_vector = np.array([x[i, j], y[i, j]])  # Shape (2,)
        output[i, j, :] = np.dot(jacobian, input_vector)  # Use np.dot instead of dot() method

# Plotting in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Use scatter to visualize the transformed points in R^3
# Color based on one of the dimensions (for instance, z-axis values)
sc = ax.scatter(output[:, :, 0], output[:, :, 1], output[:, :, 2], c=output[:, :, 2], cmap='viridis')

# Add color bar and labels
plt.colorbar(sc, label='Z-axis Value')
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
ax.set_title('3D Visualization of Jacobian Transformation')
plt.show()


C:\Users\waqar\AppData\Local\Temp\ipykernel_25584\656083419.py:15: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  output[i, j, :] = np.dot(jacobian, input_vector)  # Use np.dot instead of dot() method


In [56]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


# Prepare grid
x, y = np.meshgrid(np.arange(num_grid_points), np.arange(num_grid_points))

# Calculate output based on Jacobian
output = np.zeros((num_grid_points, num_grid_points, 3))
for i in range(num_grid_points):
    for j in range(num_grid_points):
        jacobian = jacobian_tensor_reshaped[i, j]  # Shape (3, 2)
        input_vector = np.array([x[i, j], y[i, j]])  # Shape (2,)
        output[i, j, :] = np.dot(jacobian, input_vector)  # Use np.dot

# Extract R, G, and B components from the output for coloring
R = output[:, :, 0]  # Use the first component for Red
G = output[:, :, 1]  # Use the second component for Green
B = output[:, :, 2]  # Use the third component for Blue

# Normalize the RGB components to be in the range [0, 1]
R = (R - np.min(R)) / (np.max(R) - np.min(R))
G = (G - np.min(G)) / (np.max(G) - np.min(G))
B = (B - np.min(B)) / (np.max(B) - np.min(B))

# Combine into an RGB array
colors = np.zeros((num_grid_points, num_grid_points, 3))
colors[..., 0] = R  # Red channel
colors[..., 1] = G  # Green channel
colors[..., 2] = B  # Blue channel

# Plotting in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Use scatter to visualize the transformed points in R^3 with custom colors
ax.scatter(output[:, :, 0], output[:, :, 1], output[:, :, 2], 
           c=colors.reshape(-1, 3),  # Flatten to 2D array for colors
           marker='o')

# Add labels and title
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
ax.set_title('3D Visualization of Jacobian Transformation with RGB Colors')
plt.show()


C:\Users\waqar\AppData\Local\Temp\ipykernel_25584\2189765389.py:15: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  output[i, j, :] = np.dot(jacobian, input_vector)  # Use np.dot


# Decision boundaries of InverseProjection CLassifier

In [57]:
import numpy as np

# Define the grid range for your input features
x_min, x_max = -1, 1  # Adjust these limits based on your feature range
y_min, y_max = -1, 1

# Create a mesh grid
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                     np.arange(y_min, y_max, 0.01))

# Flatten the grid to pass to the model
grid_points = np.c_[xx.ravel(), yy.ravel()]  # shape (N, 2) for 2D input
grid_points.shape


(40000, 2)

In [58]:
import torch

# Convert grid points to a tensor
grid_tensor = torch.FloatTensor(grid_points)

# Make predictions (assuming your model takes inputs in the right format)
with torch.no_grad():
    predictions = inverse_model(grid_tensor)  # Adjust based on your model's output
    predictions = torch.argmax(predictions, dim=1)  # Get class indices if using softmax output


In [59]:
# Reshape predictions to the shape of the grid
Z = predictions.numpy().reshape(xx.shape)  # Shape (N, 2) -> (100, 100) for contour plotting
Z.shape

(200, 200)

In [60]:
import matplotlib.pyplot as plt

# Plot the decision boundary
plt.figure(figsize=(10, 6))
plt.contourf(xx, yy, Z, alpha=0.8, cmap='coolwarm')  # Fill the contours

# Optionally, plot the training points
# plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, edgecolors='k', marker='o')  # X_train and y_train are your original data
plt.title("Decision Boundaries of Inverse Projection Classifier")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.colorbar(label='Class Label')
plt.show()


# Visualizing DB in 3D

In [61]:
import numpy as np

# Define the grid range for your input features
x_min, x_max = -1, 1  # Adjust these limits based on your feature range
y_min, y_max = -1, 1

# Create a mesh grid
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                     np.arange(y_min, y_max, 0.01))

# Flatten the grid to pass to the model
grid_points = np.c_[xx.ravel(), yy.ravel()]  # shape (N, 2) for 2D input


In [62]:
import torch

# Convert grid points to a tensor
grid_tensor = torch.FloatTensor(grid_points)

# Make predictions (assuming your model takes inputs in the right format)
with torch.no_grad():
    predictions = inverse_model(grid_tensor)  # Adjust based on your model's output



In [63]:
# Reshape predictions to get X, Y, Z coordinates
predictions = predictions.numpy()  # Convert to NumPy array if needed
Z = predictions.reshape(xx.shape[0], predictions.shape[1])  # Make sure this matches your grid shape


ValueError: cannot reshape array of size 120000 into shape (200,3)

In [ ]:
import numpy as np

# Assuming your model outputs predictions with shape (N, 3)
# where N is the number of grid points.

# Example predictions (replace this with your actual predictions)
# predictions.shape should be (num_grid_points, 3)
# For example: predictions = np.random.rand(xx.size, 3)

# #Flatten the original xx and yy to get X and Y coordinates
# X = xx.ravel()  # Corresponding X-coordinates for the input grid
# Y = yy.ravel()  # Corresponding Y-coordinates for the input grid

# The predictions array is already of shape (num_grid_points, 3), which is:
# predictions[:, 0] -> X-coordinate in 3D
# predictions[:, 1] -> Y-coordinate in 3D
# predictions[:, 2] -> Z-coordinate in 3D

# Extract Z-coordinates from the predictions
X = predictions[:, 0]  # This will be the third dimension from the predictions
Y = predictions[:, 1]  # This will be the third dimension from the predictions
Z = predictions[:, 2]  # This will be the third dimension from the predictions

# Now you can plot using these coordinates


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot of the predictions in 3D
scatter = ax.scatter(X, Y, Z, c='b', alpha=0.6)  # Use 'c' to specify color or use class labels

# Customize the plot
ax.set_title("3D Decision Boundaries of Inverse Projection Classifier")
ax.set_xlabel("Feature 1 (Input X)")
ax.set_ylabel("Feature 2 (Input Y)")
ax.set_zlabel("Output Feature (Z)")

# Show the plot
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Sample predictions (for demonstration; replace with your actual predictions)
num_grid_points = 100  # This should match your actual number of predictions
predictions = np.random.rand(num_grid_points, 3)  # Shape (num_grid_points, 3)

# Create example classes for color coding (3 classes)
classes = np.random.randint(0, 3, size=num_grid_points)  # Randomly assigning class labels (0, 1, or 2)

# Define a color map based on the class labels
color_map = {0: 'red', 1: 'blue', 2: 'green'}
colors = [color_map[c] for c in classes]  # Assign colors based on the classes

# Flatten the original predictions to get X, Y, and Z coordinates
X = predictions[:, 0]  # X-coordinates from the predictions
Y = predictions[:, 1]  # Y-coordinates from the predictions
Z = predictions[:, 2]  # Z-coordinates from the predictions

# Check the shapes and lengths for debugging
print(f'X shape: {X.shape}, Y shape: {Y.shape}, Z shape: {Z.shape}, Colors length: {len(colors)}')

# Check individual values for better understanding
print("X:", X)
print("Y:", Y)
print("Z:", Z)
print("Classes:", classes)

# Ensure all dimensions match
assert X.shape[0] == Y.shape[0] == Z.shape[0] == len(colors), "Shapes do not match!"

# Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot of the predictions in 3D with color coding
scatter = ax.scatter(X, Y, Z, c=colors, alpha=0.6)  # Use 'c' to specify colors

# Customize the plot
ax.set_title("3D Decision Boundaries of Inverse Projection Classifier")
ax.set_xlabel("Feature 1 (Input X)")
ax.set_ylabel("Feature 2 (Input Y)")
ax.set_zlabel("Output Feature (Z)")

# Create a legend for the classes
red_patch = plt.Line2D([0], [0], marker='o', color='w', label='Class 0', markerfacecolor='red', markersize=10)
blue_patch = plt.Line2D([0], [0], marker='o', color='w', label='Class 1', markerfacecolor='blue', markersize=10)
green_patch = plt.Line2D([0], [0], marker='o', color='w', label='Class 2', markerfacecolor='green', markersize=10)
ax.legend(handles=[red_patch, blue_patch, green_patch])

# Show the plot
plt.show()
